# SUERC AMS Cathode Data Generator

## How to use this script

Below this text is a large block of code, below that should be a GUI of some sort. If the GUI is not visible, Run the notebook by clicking "Run" in the top tool bar and wait a few seconds. 

The script requires an absolute file path for an AMSEK or NERC Sample Submission Form. This can be done by manually typing the file path into the code; however, a more convenient way is to use an ipywidgets.Button that incorporates elements of the tkinter GUI module. The result is a Yellow/Orange button that allows the user to select a file from their local drive; the file path is then automatically passed to a dataframe and processed using pandas. 

###### Warning - The GUI allows the user to select multiple files and stores the respective paths as a list. Selecting multiple files at this stage in development may cause unusual behaviour.

Once a submission form file has been chosen, select the appropriate submitter lab from the drop down menu, then hit "Upload Form". The form is processed into a "computer friendly" dataframe and is then displayed below.

###### Before the Run List or Cathode List can be generated, the g-numbers list must be created.
To do this, enter the number you wish to start from in the text entry box. You do not need to include the prefix "g", this will be added automatically. Hit the "Generate g-numbers" button. Depending on the submitter lab selected, a list of g-numbers will be generated, the standards will be identified and the appropriate suffix added to the g-number. 

###### Warning - The methods used to identify standards rely on consistent nomenclature and notation, any variations will be missed. The code can be updated to include additional/alternative notations and standards, however, variation is the bane of automation.
NB: Samples/Standards with no unique notations are impossible to resolve - I'm looking at you bbm/bk LMA BULK Tiri Barley Mash.

#### Get Files
The Delta File can be created without the need for g-numbers. Clicking the "Get Delta File" button will generate the DeltaFile, display it on the screen, and will automatically save a copy to the location where this script is being run from. 

The Cathode List and Run List will only generate if the g-numbers have been created first. At this time, only the Cathode List can be created, and this is still lacking some features. The Run List has yet to be coded in...

#### To Do List:
Generate RunList - Determine options 
Method idea: The Runlist can be broken down into 4 dataframes: Header code, Cat list, Run list, footer code.
The header code df can be mostly generated from dropdown menus. Cat list is generated from existing data. Run list needs to be generated, again GUI input can be used to offer customisation. The footer code df looks to be drawn from existing data.  

add header information to cathode list - Title, Source, Submitter Lab Name, Date.

Add group "Combiners"; unknowns are bracketed by standards some of which fall into two groups and have the notation "Group 1+2"




In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


In [11]:
import traitlets
from ipywidgets import widgets, Layout
from IPython.display import display
from tkinter import Tk, filedialog
import pandas as pd
from IPython.display import clear_output

#This section sets up the Jupyter Button Widget with the tkinter traits 
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select File"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True)

        b.description = "File Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"
# The "Select File" Button functionality is defined above but the button itself is created much later 
        
#This function processes the Excel files for the AMSEK style sheets
        
def process_amsek():
    #Reads the Excel file from absolute path, returned by "Select File" button. 
    xls = pd.ExcelFile(file_button.files[0])  
    #xls = pd.ExcelFile(r'C:\Users\Thomas\Downloads\AMSEK200306BatchNine.xls')  

    # Returns Excel sheet names as list - Might be useful if the number of sheets vary. 
    x = xls.sheet_names
    # Creates raw dataframes for each sheet - As the number of sheets is fixed, single commands are ok.
    df1 = pd.read_excel(xls, 0, skiprows=1)
    df2 = pd.read_excel(xls, 1, skiprows=1)
    df3 = pd.read_excel(xls, 2, skiprows=1)
    df4 = pd.read_excel(xls, 3, skiprows=1)
    df5 = pd.read_excel(xls, 4, skiprows=1) # For some reason the "Front" page is last

    #creates a list of sheets - the "Front" sheet is not included but can be treated seperately
    sheets = [df1, df2, df3, df4]
    #Concatenate sheets into single dataframe
    SampleList = pd.concat(sheets)
    #Removes rows that do not have a Sample ID - i.e. footer text of each sheet
    SampleList = SampleList.dropna(subset=['Sample ID²'])
    #Fills in missing group data i.e. replaces NaN with whatever is above it.  
    SampleList['Gr¹'].fillna(method='ffill', inplace = True)
    #Sets Pandas display options to show full dataframes rather than truncated dataframes
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    #Show the requested dataframe
    print('There are', (134 - len(SampleList)), 'unused cathode positions.')
    #Reset the indesing as concatenation causes repeats
    SampleList.reset_index(inplace = True, drop=True) 
    #Show the uploaded data in a tidy way
    display(SampleList)
    #Creates an Atribute for this Function to be used to call the Dataframe 
    process_amsek.dataout = SampleList
    
#The following function processes the NERC style submission forms   
def process_nerc():
    #Reads the Excel file from absolute path. Prefix path with 'r' to prevent unicode error with "\U"
    xls = pd.ExcelFile(file_button.files[0])  
    # Returns Excel sheet names as list - Might be useful if the number of sheets vary. 
    x = xls.sheet_names
    # Creates raw dataframes for each sheet - As the number of sheets is fixed, single commands are ok.
    df1 = pd.read_excel(xls, 0, skiprows=1) #Front Page
    df2 = pd.read_excel(xls, 1, skiprows=1) #Page 2
    df3 = pd.read_excel(xls, 2, skiprows=1) #Page 3
    df4 = pd.read_excel(xls, 3, skiprows=1) #Page 4
    df5 = pd.read_excel(xls, 4, skiprows=1) #Page 5

    #creates a list of sheets - the "Front" sheet is not included but can be treated seperately
    sheets = [df2, df3, df4, df5]
    #Concatenate sheets into single dataframe
    SampleList = pd.concat(sheets)
    #Removes rows that do not have a Sample ID - i.e. footer text of each sheet
    SampleList = SampleList.dropna(subset=['Sample ID2'])
    SampleList = SampleList.dropna(subset=['No. '])
    #Fills in missing group data i.e. replaces NaN with whatever is above it.  
    SampleList['Gr1'].fillna(method='ffill', inplace = True)
    #Drops columns that do not have a header
    SampleList.drop(SampleList.columns[SampleList.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    #Displays full dataframe without truncation 
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    #Sets Pandas display options to show full dataframes rather than truncated dataframes
    #pd.set_option("display.max_rows", None, "display.max_columns", None)
    #Show the requested dataframe
    print('There are', (134 - len(SampleList)), 'unused cathode positions.')
    #Reset the indesing as concatenation causes repeats
    SampleList.reset_index(inplace = True, drop=True) 
    #Show the uploaded data in a tidy way
    display(SampleList)
    #Creates an Atribute for this Function to be used to call the Dataframe 
    process_nerc.dataout = SampleList

#Function for passing submission forms to correct processing method    
def process_clicked(b):
    clear_output()
    display(tab)
    if menu.value == 1:
        process_amsek()
    else:
        process_nerc()

count = 0 #tracks the number of user created cathode entries
def insert_clicked(b):
    clear_output()
    display(tab)
    global count
    #Insert extra rows to the AMSEK style form
    if menu.value == 1:
        if count < 1:
            data = process_amsek.dataout
        else:
            data = insert_clicked.dataout
        if position_select.value == 0:
            line = pd.DataFrame({'No.':'test', 'Gr¹':group.value, 'Sample ID²':sample_type.value,
                             'Material':material.value, 'δ¹³C (‰)':offline_d13C.value, 'σ (δ¹³C)':' ',
                             'Comments³':'User Added Entry', 'Cost code⁴':' '}, index=[position_select.value-1])
        else:
            line = pd.DataFrame({'No.':'test', 'Gr¹':group.value, 'Sample ID²':sample_type.value,
                             'Material':material.value, 'δ¹³C (‰)':offline_d13C.value, 'σ (δ¹³C)':' ',
                             'Comments³':'User Added Entry', 'Cost code⁴':' '}, index=[position_select.value])
        data = data.append(line, ignore_index=False)
        data = data.sort_index().reset_index(drop=True)
        count = count+1
        print(count, " New Cathode Entries Added. There are: ",(134-len(data)), " Available Positions Remaining")
        data['No.'] = data.index
        display(data)
        insert_clicked.dataout = data
    #Insert rows to the NERC style forms
    else:
        if count < 1:
            data = process_nerc.dataout
        else:
            data = insert_clicked.dataout
        if position_select.value == 0:
            line = pd.DataFrame({'No. ':'test', 'Gr1':group.value, 'Sample ID2':sample_type.value,
                             'Material':material.value, 'd13C (‰)':offline_d13C.value, 's (d13C)':' ',
                             'Comments3':'User Added Entry', 'Cost code4':' ', 'Project No.':' ',
                                'Total vol CO2 (ml)5':' ', 'Low C weight (mg)6':' ','Estimated Age':' '},
                                index=[position_select.value-1])
        else:
            line = pd.DataFrame({'No. ':'test', 'Gr1':group.value, 'Sample ID2':sample_type.value,
                             'Material':material.value, 'd13C (‰)':offline_d13C.value, 's (d13C)':' ',
                             'Comments3':'User Added Entry', 'Cost code4':' ', 'Project No.':' ',
                                'Total vol CO2 (ml)5':' ', 'Low C weight (mg)6':' ','Estimated Age':' '},
                                index=[position_select.value])
        data = data.append(line, ignore_index=False)
        data = data.sort_index().reset_index(drop=True)
        count = count+1
        print(count, " New Cathode Entries Added. There are: ",(134-len(data)), " Available Positions Remaining")
        data['No. '] = data.index
        display(data)
        insert_clicked.dataout = data
    
def deltafile_clicked(b):
    clear_output()
    display(tab)
    global count
    if menu.value == 1:
        if count > 0 :
            deltafile = pd.concat([insert_clicked.dataout['Sample ID²'], insert_clicked.dataout['δ¹³C (‰)']], axis=1)
        else:
            deltafile = pd.concat([process_amsek.dataout['Sample ID²'], process_amsek.dataout['δ¹³C (‰)']], axis=1)
        deltafile['Sigma']=0.5
        deltafile.rename(columns={'Sample ID²': 'SampleType', 'δ¹³C (‰)': 'Delta13C'}, inplace=True)
        print('Deltafile has been saved to the directory this code was run from...')
        display(deltafile)
        deltafile.to_csv('Deltafile',sep='\t', index=False)
    else:
        if count > 0:
            deltafile = pd.concat([insert_clicked.dataout['Sample ID2'], insert_clicked.dataout['d13C (‰)']], axis=1)
        else:
            deltafile = pd.concat([process_nerc.dataout['Sample ID2'], process_nerc.dataout['d13C (‰)']], axis=1)
        deltafile['Sigma']=0.5
        deltafile.rename(columns={'Sample ID2': 'SampleType', 'd13C (‰)': 'Delta13C'}, inplace=True)
        print('Deltafile has been saved to the directory this code was run from...')
        display(deltafile)
        deltafile.to_csv('Deltafile',sep='\t', index=False)
    
def geng_clicked(b):
    clear_output()
    display(tab)
    global count
    g_int = int(g_start.value)
    if menu.value == 1:
        #Have any lines been inserted to the cathode list? 
        if count > 0:
            data = insert_clicked.dataout
        else:
            data = process_amsek.dataout
        length = len(data)
        g_numbers = pd.DataFrame(range(g_int, g_int + length), columns = ['g-numbers'])
        #creates a list of numbers in the g range  
        g_numbers['g-numbers'] = 'g' + g_numbers['g-numbers'].astype(str)
        
        for i in range(len(data)) : 
            #Find M samples and give suffix -ox
            if data.loc[i,"Sample ID²"].find('M') == 0 and data.loc[i,"Sample ID²"].find('B') != 1:
                g_numbers.loc[i, 'g-numbers'] = g_numbers.loc[i, 'g-numbers'] + '-ox'
            #Find MB samples and gives suffix - mb
            if data.loc[i,"Sample ID²"].find('M') == 0 and data.loc[i,"Sample ID²"].find('B') == 1:
                g_numbers.loc[i, 'g-numbers'] = g_numbers.loc[i, 'g-numbers'] + '-mb'
            #Finds BK Samples and gives suffix -bk
            if data.loc[i,"Sample ID²"].find('B') == 0 and data.loc[i,"Sample ID²"].find('K') == 1:
                g_numbers.loc[i, 'g-numbers'] = g_numbers.loc[i, 'g-numbers'] + '-bk'
            #Finds HA Samples and gives suffix -ha
            if data.loc[i,"Sample ID²"].find('H') == 0:
                g_numbers.loc[i, 'g-numbers'] = g_numbers.loc[i, 'g-numbers'] + '-ha'
            #Finds BBM Samples and gives suffix -bbm
            if data.loc[i,"Sample ID²"].find('B') == 0 and data.loc[i,"Sample ID²"].find('M') == 2:
                g_numbers.loc[i, 'g-numbers'] = g_numbers.loc[i, 'g-numbers'] + '-bbm'
            #Finds WNM Samples and gives suffix -nms
            if data.loc[i,"Sample ID²"].find('W') == 0:
                g_numbers.loc[i, 'g-numbers'] = g_numbers.loc[i, 'g-numbers'] + '-nms'
                #Finds UC Samples and gives suffix -uc
            if data.loc[i,"Sample ID²"].find('U') == 0:
                g_numbers.loc[i, 'g-numbers'] = g_numbers.loc[i, 'g-numbers'] + '-uc'
        #geng_clicked.dataout = g_numbers
        
    else:
        if count > 0:
            data = insert_clicked.dataout
        else:
            data = process_nerc.dataout
        length = len(data)
        #creates a list of numbers in the g range 
        g_numbers = pd.DataFrame(range(g_int, g_int + length), columns = ['g-numbers'])
        #Prefix each item with "g"
        g_numbers['g-numbers'] = 'g' + g_numbers['g-numbers'].astype(str)
        #For loop searches through "Material" column looking for items in the Dictionary standards_dict
        #and automatically adds the appropriate suffix
        for i in range(len(data)) :
            if str(data["Material"].iloc[i]) in standards_dict:
                g_numbers.loc[i, 'g-numbers'] = g_numbers.loc[i, 'g-numbers'] + standards_dict[str(data["Material"].iloc[i])]
        
    print("Range: " +str(g_numbers.loc[0, "g-numbers"])+ " To " +str(g_numbers["g-numbers"].iloc[-1])+ " Generated" )
#str(g_numbers.tail(1))
    display(g_numbers)
    geng_clicked.dataout = g_numbers
    
    
#Dictionary for g number suffix - ONLY EXACT MATCHES WILL WORK, UPDATE DICT WITH ALTERNATIVES
standards_dict = {
  "LMA NIST Oxalic Acid II" : "-ox",
  "LMA  BULK Tiri Barley Mash" : "-bk",
  "LMA BULK Belfast Cellulose" : "-ha",
  "AP1 CO2": "-ha",
  "AP1CO2": "-ha",
  "TIRI Barleymash": "-ha",
  "Bulk Iceland Spar Calcite": "-uc",
  "Ultracarbon - new": "-uc"
} 

def cathode_clicked(b):
    #Reset out put to keep from getting messy
    clear_output()
    display(tab)
    global count
    #If AMSEK form - 
    if menu.value == 1:
        if count > 0 :     
            catdata = insert_clicked.dataout
        else:
            catdata = process_amsek.dataout
        #Concatenate the required columns including G-numbers
        cathodelist = pd.concat((catdata["No."],catdata["Sample ID²"],catdata["Material"],
                               catdata["δ¹³C (‰)"], geng_clicked.dataout["g-numbers"],
                               catdata["Gr¹"]), axis=1)
        #Rename them to match convention
        cathodelist.rename(columns={'No.': 'Position','Sample ID²': 'SampleType (Preparation ID)',
                                    'δ¹³C (‰)': 'Off-line d¹³C (‰)', 'g-numbers':'Sample Name (Measurement ID)',
                                   'Gr¹':'Group'}, inplace=True)
        display(cathodelist)
        cathode_clicked.dataout = cathodelist
        #Export to Excel file in location where this code is run from
        cathodelist.to_excel('AMSEK Cathode List.xlsx', index=False, startrow=3)
    else:
        if count > 0 :
            catdata = insert_clicked.dataout
        else:
            catdata = process_nerc.dataout
        #See above for comments
        cathodelist = pd.concat((catdata["No. "],catdata["Sample ID2"],catdata["Material"],
                               catdata["d13C (‰)"], geng_clicked.dataout["g-numbers"],
                               catdata["Gr1"]), axis=1)
        cathodelist.rename(columns={'No. ': 'Position','Sample ID2': 'SampleType (Preparation ID)',
                                    'd13C (‰)': 'Off-line d¹³C (‰)', 'g-numbers':'Sample Name (Measurement ID)',
                                   'Gr1':'Group'}, inplace=True)
        display(cathodelist)
        cathode_clicked.dataout = cathodelist
        cathodelist.to_excel('NERC Cathode List.xlsx', index=False, startrow=3)

def runlist_clicked(b):
    clear_output()
    display(tab)
    #Create Batch Control Dataframe
    data = {'B':['isotope','source','park','mode','autorange','judge','parkmode'],
            'C':[run_options1.value, run_options2.value, run_options3.value,
                run_options4.value, run_options5.value, run_options6.value,
                run_options7.value]}
    batchcontroldf = pd.DataFrame(data)
    batchcontroldf.insert(0, 'A', '    batch')
    display(batchcontroldf)
    
    #Here is where the IF statement needs to start
    #Create Cat data frame
    catdf = cathode_clicked.dataout
    catdf.drop(['Material','Off-line d¹³C (‰)','Group'], axis=1, inplace=True)
    catdf.insert(0, 'cat', '    cat')
    catdf.insert(4, 'Name', '1')
    display(catdf)
    
    
    
    #Create Run List File
    f = open("RunList.txt", "w",  encoding="utf-8")
    f.write("# \n#----Batch Control Items---- \n#\n")
    batchcontroldf.to_csv(f, sep='\t', header=False, index=False)
    f.write("\n# direc   pos   Smtype   Sample   Name1   Name2 \n")
    f.write("#===================================================\n")
    catdf.to_csv(f, sep='\t', header=False, index=False)
    f.write("\n# direce Item Pos Grp Sum Runs Md Tlimit Climit Warm MinRuns MinErr \n")
    f.write("#====================================================================\n")
    f.close()
    
    print("Done")
    
#Shows full text in GUI and defines button size
style = {'description_width': 'initial'}
layout = Layout(width='300px', height='30px')
layout2 = Layout(width='200px', height='50px')
layout3 = Layout(width='800px')
#Select submission form type: AMSEK or NERC
menu = widgets.Dropdown(
       options=[('AMSEK Submission Form',1), ('NERC Submission Form',2)],
       value=1,
       description='Submitter Lab:', style=style)
#Button to process the input file
process_button = widgets.Button(description='Upload Form')
#File select button        
file_button = SelectFilesButton()
#What to do when "Process Form" is clicked
process_button.on_click(process_clicked)
#Field to enter starting g-number
g_start = widgets.Text(value='', description='g-numbers start from: g', style=style )
#Button to generate cathode list
cathodelist_button = widgets.Button(description='Get Cathode List', layout=layout2)
cathodelist_button.on_click(cathode_clicked)
#Button to generate Deltafile
deltafile_button = widgets.Button(description='Get Delta File', layout =layout2)
deltafile_button.on_click(deltafile_clicked)
#Button to generate Runlist
runlist_button = widgets.Button(description='Get Run List', layout=layout2)
runlist_button.on_click(runlist_clicked)
#Generate g-numbers Button
geng_button = widgets.Button(description='Generate g-numbers', layout=layout)
geng_button.on_click(geng_clicked)

#CATHODE LIST OPTIONS 
position_select = widgets.IntSlider(value=0, min=0,
                                    max=133, step=1,
                                    description='Insert at Position:',
                                    disabled=False,
                                    continuous_update=False, 
                                    orientation='horizontal',
                                    readout=True,
                                    readout_format='d',
                                    style={'description_width': 'initial'},
                                   layout=layout3)
sample_type = widgets.Text(value='', description='Sample Type:', style=style )
material = widgets.Text(value='', description='Material:', style=style )
offline_d13C = widgets.Text(value='', description='Off-line d¹³C (‰):', style=style )
group = widgets.Text(value='', description='Group:', style=style)
insert_button = widgets.Button(description='Insert Cathode Entry', layout=layout)
insert_button.on_click(insert_clicked)

#RUNLIST OPTIONS DROPDOWN MENU
run_options1 = widgets.Dropdown(
       options=[('C14')],
       value='C14',
       description='Isotope:', style=style)
run_options2 = widgets.Dropdown(
       options=[('S1'), ('S2')],
       value='S1',
       description='Source:', style=style)
run_options3 = widgets.Dropdown(
       options=[('0')],
       value='0',
       description='Park:', style=style)
run_options4 = widgets.Dropdown(
       options=[('nrm')],
       value='nrm',
       description='Mode:', style=style)
run_options5 = widgets.Dropdown(
       options=[('no')],
       value='no',
       description='Autorange:', style=style)
run_options6 = widgets.Dropdown(
       options=[('on')],
       value='on',
       description='Judge:', style=style)
run_options7 = widgets.Dropdown(
       options=[('on')],
       value='on',
       description='Park mode:', style=style)
dropdown1 = widgets.VBox([run_options1, run_options2, run_options3,
                           run_options4, run_options5, run_options6,
                           run_options7])

run_options8 = widgets.Dropdown(
       options=[('8')],
       value='8',
       description='Default Runs:', style=style)
run_options9 = widgets.Dropdown(
       options=[('C')],
       value='C',
       description='Default Md:', style=style)
run_options10 = widgets.Dropdown(
       options=[('4000')],
       value='4000',
       description='Default Tlimit:', style=style)
run_options11 = widgets.Dropdown(
       options=[('40000')],
       value='40000',
       description='Default Climit:', style=style)
run_options12 = widgets.Dropdown(
       options=[('300')],
       value='300',
       description='Default Warm:', style=style)
run_options13 = widgets.Dropdown(
       options=[('3')],
       value='3',
       description='Default MinRuns:', style=style)
run_options14 = widgets.Dropdown(
       options=[('0.3')],
       value='0.3',
       description='Default MinErr:', style=style)
dropdown2 = widgets.VBox([run_options8, run_options9, run_options10,
                           run_options11, run_options12, run_options13,
                           run_options14])
dropdownbox = widgets.HBox([dropdown1, dropdown2])

#GUI containers 
box1 = widgets.HBox([file_button, menu, process_button])
box2 = widgets.HBox([g_start, geng_button])
box3 = widgets.HBox([deltafile_button])
box4 = widgets.HBox([runlist_button])
box5 = widgets.HBox([cathodelist_button])
box6 = widgets.VBox([sample_type, material, offline_d13C, group, insert_button])
mainbox = widgets.VBox([box1])
deltabox = widgets.VBox([box3])
runlistbox = widgets.VBox([dropdownbox, box4])
cathodebox = widgets.VBox([position_select, box6, box2, box5])

children = [mainbox, cathodebox, deltabox, runlistbox]
tab = widgets.Tab()
tab.children = children
tab.set_title(0, 'Main')
tab.set_title(1, 'Cathode List')
tab.set_title(2, 'Delta File')
tab.set_title(3, 'Run List')
tab

In [18]:
print(insert_clicked.dataout)

     No.          Gr¹ Sample ID²             Material δ¹³C (‰) σ (δ¹³C)  \
0      0      Group 1      M8562            Oxalic II    -17.6      NaN   
1      1      Group 1     BK4950                 Wood    -20.2      NaN   
2      2      Group 1     HA6123           Humic Acid    -28.6      NaN   
3      3  Group SUERC         UG        Ultragraphite      -10            
4      4      Group 1    GU55281     carbonate, coral     -2.2      NaN   
5      5      Group 1    GU55282     carbonate, coral     -1.2      NaN   
6      6      Group 1    GU55283     carbonate, coral     -2.7      NaN   
7      7      Group 1    GU55284     carbonate, coral    -16.5      NaN   
8      8      Group 1    GU55285     carbonate, coral    -16.5      NaN   
9      9      Group 1    GU55286     carbonate, coral      -25      NaN   
10    10      Group 1    GU55287     carbonate, coral    -16.3      NaN   
11    11      Group 2      M8563            Oxalic II    -17.3      NaN   
12    12      Group 2    